In [1]:
import pathlib

import pandas as pd
import tqdm

In [2]:
CHUNKSIZE = 1e5

base_df = pd.read_csv(
    "../data/База по всем организациям и ИП.zip",
    sep=";", 
    chunksize=CHUNKSIZE, 
    dtype=str
)

In [11]:
out_file = pathlib.Path("/home/pavel/all_companies.csv")

for chunk in tqdm.tqdm(base_df):
    chunk = chunk.loc[chunk["ОКОПФ (расшифровка)"] != "Индивидуальный предприниматель"]
    chunk["name"] = chunk["Наименование полное"].str.strip().str.upper()
    chunk["tax_number"] = chunk["ИНН"]
    chunk = chunk[["name", "tax_number"]]

    if out_file.exists():
        chunk.to_csv(out_file, header=False, index=False, mode="a")
    else:
        chunk.to_csv(out_file, header=True, index=False, mode="w")


247it [10:09,  2.47s/it]


In [3]:
mod = pd.read_csv("../data/opendata/models.csv")

/tmp/ipykernel_13449/2403228113.py:1: DtypeWarning: Columns (0,12,15,21,23,27,29,35) have mixed types. Specify dtype option on import or set low_memory=False.
  mod = pd.read_csv("../data/opendata/models.csv")


In [18]:
def is_company(row):
    row = row.fillna("")
    if row["authors"] == row["patent holders"]:
        return False
    
    for name in list(map(str.strip, row["patent holders"].split("\n"))):
        name_parts = list(filter(lambda x: len(x) > 0, map(str.strip, name.split(" "))))
        if (
            len(name_parts) == 3
            and all(part[0].isupper() for part in name_parts)
        ):
            return False
        if (
            len(name_parts) == 2 
            and name_parts[0][0].isupper()
            and name_parts[1].replace(".", "").isupper()
        ):
            return False
        
    return True

In [22]:
mod["company"] = mod.apply(is_company, axis=1)
mod[["patent holders", "company"]].head(50)

,patent holders,company
0,Напетваридзе С.О. (RU)\r\nЧалаганидзе Шота Ива...,True
1,Дронов Владимир Дмитриевич\r\nКолобаев Владими...,False
2,"Малое производственное предприятие ""Пирс""",True
3,Селиванов Виктор Викторович,False
4,Васильев Михаил Григорьевич,False
5,Васильев Михаил Григорьевич,False
6,Васильев Михаил Григорьевич,False
7,Селиванов Вениамин Васильевич,False
8,"Товарищество с ограниченной ответственностью ""...",True
9,Чистяков Игорь Викторович,False


In [34]:
mod_to_merge = mod.loc[(
    mod["patent holders"].str.contains("RU", na=False)
    ), 
    ["registration number", "patent holders", "company"]
]
mod_to_merge.columns = ["reg_num", "name", "company"]
mod_to_merge["name"] = mod_to_merge["name"].str.replace("(RU)", "").str.strip().str.upper()

mod_to_merge

,reg_num,name,company
0,﻿1,НАПЕТВАРИДЗЕ С.О. \r\nЧАЛАГАНИДЗЕ ШОТА ИВАНОВИ...,True
192,193,"ЗАКРЫТОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО ""ПАРТНЕР""",True
203,204,"ХОЗРАСЧЕТНЫЙ ЦЕНТР ""ИНТЕГРАЛ""",True
232,234,ЧАБУРОВ ВИКТОР ФЕДОРОВИЧ (LT)\r\nЧАБУРОВ АНДРЕ...,False
352,354,"ЗАКРЫТОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО ""РОССИЙСКО-ЧЕШСК...",True
...,...,...,...
222356,225645,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""РМ РЕЙЛ АБАКАНВАГОНМАШ"" ...",True
222357,225646,БРАТЧИКОВА ДИНАРА МАРАТОВНА,True
222358,225647,МАЛОВИЧКО ВЛАДИМИР ВАЛЕНТИНОВИЧ,False
222359,225648,КИРИЛЛОВ ВЛАДИМИР ИЛЬИЧ,False


In [6]:
companies_to_merge = pd.read_csv("/home/pavel/all_companies.csv")

/tmp/ipykernel_13449/2703025525.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  companies_to_merge = pd.read_csv("/home/pavel/all_companies.csv")


In [38]:
companies_to_merge.loc[companies_to_merge["name"] == "ЗУБОВА НАТАЛЬЯ БОРИСОВНА"]

,name,tax_number
2539878,ЗУБОВА НАТАЛЬЯ БОРИСОВНА,524800502839.0
3982214,ЗУБОВА НАТАЛЬЯ БОРИСОВНА,772421742044.0
4469190,ЗУБОВА НАТАЛЬЯ БОРИСОВНА,222402990913.0
14677707,ЗУБОВА НАТАЛЬЯ БОРИСОВНА,614107780410.0
16059686,ЗУБОВА НАТАЛЬЯ БОРИСОВНА,662301249945.0


In [35]:
merged1 = mod_to_merge.merge(
    companies_to_merge,
    how="left",
    on="name"
).drop_duplicates("reg_num")

In [36]:
merged1.head(50)

,reg_num,name,company,tax_number
0,﻿1,НАПЕТВАРИДЗЕ С.О. \r\nЧАЛАГАНИДЗЕ ШОТА ИВАНОВИ...,True,NaN
1,193,"ЗАКРЫТОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО ""ПАРТНЕР""",True,5190110664
70,204,"ХОЗРАСЧЕТНЫЙ ЦЕНТР ""ИНТЕГРАЛ""",True,NaN
71,234,ЧАБУРОВ ВИКТОР ФЕДОРОВИЧ (LT)\r\nЧАБУРОВ АНДРЕ...,False,NaN
72,354,"ЗАКРЫТОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО ""РОССИЙСКО-ЧЕШСК...",True,7701008480.0
73,419,"ООО ""НИАЛ""",True,7806039888.0
74,424,"ООО ""НИАЛ""",True,7806039888.0
75,565,КОВАЛЕНКО НИКОЛАЙ ВЛАДИМИРОВИЧ \r\nШАМИС ВАЛЕР...,False,NaN
76,581,"АКЦИОНЕРНОЕ ОБЩЕСТВО ЗАКРЫТОГО ТИПА ""ЛИТЭКС"" \...",False,NaN
77,835,БОРЩ-КОМПОНИЕЦ ВИТАЛИЙ ИВАНОВИЧ,True,NaN


In [39]:
unmerged1 = merged1.loc[merged1["tax_number"].isna()]
unmerged1

,reg_num,name,company,tax_number
0,﻿1,НАПЕТВАРИДЗЕ С.О. \r\nЧАЛАГАНИДЗЕ ШОТА ИВАНОВИ...,True,NaN
70,204,"ХОЗРАСЧЕТНЫЙ ЦЕНТР ""ИНТЕГРАЛ""",True,NaN
71,234,ЧАБУРОВ ВИКТОР ФЕДОРОВИЧ (LT)\r\nЧАБУРОВ АНДРЕ...,False,NaN
75,565,КОВАЛЕНКО НИКОЛАЙ ВЛАДИМИРОВИЧ \r\nШАМИС ВАЛЕР...,False,NaN
76,581,"АКЦИОНЕРНОЕ ОБЩЕСТВО ЗАКРЫТОГО ТИПА ""ЛИТЭКС"" \...",False,NaN
...,...,...,...,...
2810612,225642,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""КРАС...",True,NaN
2810617,225644,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ИЖЕВ...",True,NaN
2810618,225645,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""РМ РЕЙЛ АБАКАНВАГОНМАШ"" ...",True,NaN
2810620,225647,МАЛОВИЧКО ВЛАДИМИР ВАЛЕНТИНОВИЧ,False,NaN


In [59]:
mod.loc[mod["patent holders"].str.contains("Рейл Абаканвагонмаш", na=False)]

,registration number,registration date,application number,application date,authors,authors in latin,patent holders,patent holders in latin,correspondence address,correspondence address in latin,...,PCT application publish number,PCT application publish date,patent grant publish date,patent grant publish number,revoked patent number,expiration date,utility model formula numbers for which patent term is prolonged,actual,publication URL,company
206208,209497,20220316.0,2.021129e+09,20211007.0,Бажин Олег Владимирович (RU)\r\nКузнецов Никол...,\r\n,"Акционерное общество ""РМ Рейл Абаканвагонмаш"" ...",NaN,"430006, Респ. Мордовия, г. Саранск, ул. Лодыги...",NaN,...,NaN,NaN,20220316.0,8.0,NaN,NaN,NaN,True,http://www1.fips.ru/fips_servl/fips_servlet?DB...,True
206981,210270,20220405.0,2.021129e+09,20211007.0,Бажин Олег Владимирович (RU)\r\nКузнецов Никол...,\r\n,Акционерное общество «РМ Рейл Абаканвагонмаш» ...,NaN,"430006, Рес. Мордовия, г. Саранск, ул. Лодыгин...",NaN,...,NaN,NaN,20220405.0,10.0,NaN,NaN,NaN,True,http://www1.fips.ru/fips_servl/fips_servlet?DB...,True
207543,210832,20220506.0,2.022102e+09,20220128.0,Бажин Олег Владимирович (RU)\r\nКузнецов Никол...,\r\n,"Акционерное общество ""РМ Рейл Абаканвагонмаш"" ...",NaN,"430006, Рес. Мордовия, г. Саранск, ул. Лодыгин...",NaN,...,NaN,NaN,20220506.0,13.0,NaN,NaN,NaN,True,http://www1.fips.ru/fips_servl/fips_servlet?DB...,True
207544,210833,20220506.0,2.022104e+09,20220217.0,Бажин Олег Владимирович (RU)\r\nКузнецов Никол...,\r\n,"Акционерное общество ""РМ Рейл Абаканвагонмаш"" ...",NaN,"430006, Рес. Мордовия, г. Саранск, ул. Лодыгин...",NaN,...,NaN,NaN,20220506.0,13.0,NaN,NaN,NaN,True,http://www1.fips.ru/fips_servl/fips_servlet?DB...,True


In [56]:
unmerged1.loc[2810618]

KeyError: 2810618

In [60]:
mod.loc[207544]

registration number                                                                                                      210833
registration date                                                                                                    20220506.0
application number                                                                                                 2022104090.0
application date                                                                                                     20220217.0
authors                                                                       Бажин Олег Владимирович (RU)\r\nКузнецов Никол...
authors in latin                                                                                                           \r\n
patent holders                                                                Акционерное общество "РМ Рейл Абаканвагонмаш" ...
patent holders in latin                                                                                 

In [54]:
companies_to_merge.loc[companies_to_merge["name"].str.contains("АБАКАНВАГОНМАШ", na=False)]

,name,tax_number
9759180,ПЕРВИЧНАЯ ОБЩЕСТВЕННАЯ ПРОФСОЮЗНАЯ ОРГАНИЗАЦИЯ...,1901022548.0
15969632,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""РМ РЕЙЛ АБАКАНВАГОНМАШ""",1901142757.0


In [40]:
tm = pd.read_csv("../data/opendata/data-20240601-structure-20180828.csv")

/tmp/ipykernel_13449/642614379.py:1: DtypeWarning: Columns (0,2,13,19,21,22,24,25,31,35,36,39,41,43,45,49,53,55) have mixed types. Specify dtype option on import or set low_memory=False.
  tm = pd.read_csv("../data/opendata/data-20240601-structure-20180828.csv")


In [41]:
tm_to_merge = tm.loc[
    tm["right holder country code"] == "RU", ["right holder name", "right holder ogrn"]
]
tm_to_merge.columns=["name", "ogrn"]
tm_to_merge["name"] = tm_to_merge["name"].str.strip().str.upper()
tm_to_merge.dropna(subset=["ogrn"], inplace=True)
tm_to_merge.drop_duplicates(subset=["ogrn"], inplace=True)
tm_to_merge

,name,ogrn
48,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""ЧЕБОКСАРСКИЙ ЭЛЕКТРОАППА...",1022101129896
56,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""КОРДИАНТ""",1027600842972
105,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""КАЗАНСКИЙ ОПТИКО-МЕХАНИЧ...",1021603617188
187,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ПИРЕ...",1117746594371
198,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""ИШИМСКИЙ МЕХАНИЧЕСКИЙ ЗА...",1087205000948
...,...,...
958114,ЯЦЕНКО ВАДИМ ВАДИМОВИЧ,322370200017840.0
958115,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ВЕРД...",1215000093021.0
958117,КИЯШКО ИЛОНА ГЕННАДЬЕВНА,307532134500080.0
958119,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ИЗДА...",1197746214160.0


In [50]:
merged2 = unmerged1.merge(
    tm_to_merge,
    how="left",
    on="name"
).drop_duplicates("reg_num")

In [52]:
merged1["tax_number"].notna().sum() + merged2["ogrn"].notna().sum()

61723